In [32]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import colorsys
import random

Colors

In [33]:
# colors = np.random.randint(0, 256, size=(50, 3), dtype=np.uint8)

def hsv_to_rgb(h, s, v):
    """
    Converts a color from HSV color space to RGB color space.
    Parameters:
        h (float): hue value in range [0, 1]
        s (float): saturation value in range [0, 1]
        v (float): value (brightness) value in range [0, 1]
    Returns:
        rgb (tuple): a tuple of integers representing the RGB color values
                     in range [0, 255]
    """
    # Convert from HSV to RGB using the colorsys library
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    # Convert the RGB values from floats in range [0, 1] to integers in range [0, 255]
    rgb = np.array([r*255, g*255, b*255], dtype=np.uint8)
    # Return the RGB color values as a tuple
    return rgb


n_colors = 50  # Define the number of colors to generate.
golden_ratio_conjugate = 0.618033988749895  # Define the golden ratio.

colors = []  # Initialize an empty list to store the generated colors.

# Loop over the number of colors to generate.
for i in range(n_colors):
    hue = i / n_colors  # Calculate the hue value.
    hue += golden_ratio_conjugate  # Add the golden ratio to the hue value.
    # Take the modulo 1 of the hue value to ensure it's in the range [0,1).
    hue %= 1
    # Convert the HSV color to RGB color.
    rgb = hsv_to_rgb(hue, 0.8, 0.95)

    # Convert the RGB color to BGR color.
    bgr = [rgb[2], rgb[1], rgb[0]]
    # Append the BGR color to the list of generated colors.
    colors.append(bgr)

random.shuffle(colors)

In [34]:
def show_image(img=[], resize=-1, title='img'):
    """
    Display an image using OpenCV.

    Parameters:
    img (numpy.ndarray): A NumPy array representing the image.
    resize (float): A float value representing the resize factor of the image.
    title (str): A string representing the title of the image window.
    """
    # Get the dimensions of the image
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else:
        height, width = img.shape

     # Resize the image if requested
    if 0 < resize < 2:
        img = cv2.resize(img, [int(width*resize), int(height*resize)])

    # Create a window for the image and display it
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



def show_image_mac(img=[], resize=-1, title='img'):
    """
    Display an image using OpenCV.

    Parameters:
    img (numpy.ndarray): A NumPy array representing the image.
    resize (float): A float value representing the resize factor of the image.
    title (str): A string representing the title of the image window.
    """
    # Get the dimensions of the image
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape

     # Resize the image if requested
    if 0 < resize < 2:
      img = cv2.resize(img, [int(width*resize), int(height*resize)])

    # Create a window for the image and display it
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)

    while True:
        k = cv2.waitKey(30) & 0xFF
        if k == 27: # wait for ESC key to exit
            cv2.destroyAllWindows()
            cv2.waitKey(1)
        elif k == ord('s'): # wait for 's' key to save and exit
            cv2.imwrite('bar.jpg', img)
            cv2.destroyAllWindows()
            cv2.waitKey(1)
        if cv2.getWindowProperty("image", 0) == -1:
            break



def plot_histogram(image):
    """
    Generate a histogram for an image using Matplotlib.

    Parameters:
    image (numpy.ndarray): A NumPy array representing an image.

    Returns:
    matplotlib.figure.Figure: A Matplotlib Figure object representing the histogram.
    """
    # Generate the histogram using np.histogram
    hist, bins = np.histogram(image.ravel(), bins=256, range=(0, 256))

    # Create a new figure
    fig = plt.figure()

    # Add a new subplot to the figure
    ax = fig.add_subplot(1, 1, 1)

    # Plot the histogram
    ax.hist(image.ravel(), bins=256, range=(0, 256))

    # Set the title and axis labels
    ax.set_title("Histogram")
    ax.set_xlabel("Intensity")
    ax.set_ylabel("Frequency")

    # Show the plot
    plt.show()

    # Return the cumulative histogram
    return hist


def cummulative_histogram(img):
    """
    Create a cumulative histogram of a grayscale image.

    Parameters:
    img (numpy.ndarray): A grayscale image represented as a NumPy array.

    Returns:
    numpy.ndarray: The cumulative histogram of the image.
    """
    # Calculate the histogram of the image
    hist, bins = np.histogram(img.ravel(), 256, [0, 256])

    # Calculate the cumulative sum of the histogram
    cum_hist = np.cumsum(hist)

    # Plot the cumulative histogram
    plt.plot(cum_hist, color='black')

    # Set the axis labels and title
    plt.xlim([0, 256])
    plt.ylim([0, np.max(cum_hist)])
    plt.xlabel('Pixel intensity')
    plt.ylabel('Cumulative frequency')
    plt.title('Cummulative Histogram')

    # Show the plot
    plt.show()

    # Return the cumulative histogram
    return cum_hist


def bgr_to_gray(img):
    """
    Convert a BGR image to grayscale using OpenCV.

    Parameters:
    img (numpy.ndarray): The input BGR image.

    Returns:
    numpy.ndarray: The grayscale image.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray


def linear_contrast_stretch(img, t0, t1):
    """
    Apply linear contrast stretching to an image using fixed thresholds.

    Parameters:
    img (numpy.ndarray): The input grayscale image.
    t0 (int): The lower threshold.
    t1 (int): The upper threshold.

    Returns:
    numpy.ndarray: The contrast-stretched image.
    """
    # Copy the input image to a new array
    stretched = img.copy()

    # Calculate the slope and intercept of the linear transformation
    k = 255 / (t1 - t0)
    b = -k * t0

    # Apply the linear transformation to the pixel values in the image
    stretched[img < t0] = 0
    stretched[img > t1] = 255
    stretched[np.logical_and(img >= t0, img <= t1)] = k * \
        img[np.logical_and(img >= t0, img <= t1)] + b

    # Convert the output image to the uint8 data type
    stretched = np.uint8(stretched)

    return stretched


def median(img):
    """
    Finds the median position of a given image.

    Parameters:
    img (numpy.ndarray): The input image.

    Returns:
    int: The median position of the histogram.
    """

    # Compute the cumulative distribution of the histogram
    cumsum = cummulative_histogram(img)

    # Find the median position of the histogram
    median_pos = np.searchsorted(cumsum, cumsum[-1] // 2)

    return median_pos


def binarize_image(img, threshold):
    """
    Binarize an input grayscale image using a given threshold value.

    Parameters:
    img (numpy.ndarray): The input grayscale image.
    threshold (int): The threshold value used for binarization.

    Returns:
    numpy.ndarray: The binarized image.
    """
    # Apply thresholding to the input image
    _, binarized = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)

    return binarized


def otsu_threshold(image):
    """
    Computes the Otsu threshold for an input image.

    Parameters:
    image (numpy.ndarray): The input image.

    Returns:
    int: The computed Otsu threshold.
    """
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the histogram of the grayscale image
    histogram, bin_edges = np.histogram(gray_image, bins=256, range=(0, 255))

    # Compute the total number of pixels in the image
    total_pixels = gray_image.shape[0] * gray_image.shape[1]

    # Compute the probabilities of each intensity value
    intensity_probabilities = histogram / total_pixels

    # Compute the cumulative sum of probabilities
    cumulative_sum_probabilities = np.cumsum(intensity_probabilities)

    # Compute the cumulative sum of intensity values
    cumulative_sum_intensities = np.cumsum(
        np.arange(256) * intensity_probabilities)

    # Compute the mean intensity value of the image
    mean_intensity = cumulative_sum_intensities[-1]

    # Initialize the maximum between-class variance to zero
    max_between_class_variance = 0

    # Initialize the threshold value to zero
    threshold = 0

    # Loop over all possible threshold values
    for i in range(256):
        # Compute the probabilities of the two classes
        class1_probability = cumulative_sum_probabilities[i]
        class2_probability = 1 - class1_probability

        # Skip threshold values for which one of the classes has no pixels
        if class1_probability == 0 or class2_probability == 0:
            continue

        # Compute the mean intensity values of the two classes
        class1_mean_intensity = cumulative_sum_intensities[i] / \
            class1_probability
        class2_mean_intensity = (
            mean_intensity - cumulative_sum_intensities[i]) / class2_probability

        # Compute the between-class variance
        between_class_variance = class1_probability * class2_probability * \
            (class1_mean_intensity - class2_mean_intensity) ** 2

        # Update the maximum between-class variance and the threshold value
        if between_class_variance > max_between_class_variance:
            max_between_class_variance = between_class_variance
            threshold = i

    return threshold


def erode(img, kernel_size=3, iterations=1):
    # Create a kernel of a given size and shape (rectangle in this case)
    kernel = cv2.getStructuringElement(
        cv2.MORPH_RECT, (kernel_size, kernel_size))
    # Erode the image using the kernel and given number of iterations
    eroded_img = cv2.erode(img, kernel, iterations=iterations)
    # Return the eroded image
    return eroded_img


def dilate(img, kernel_size=3, iterations=1):
    # Create a kernel of a given size and shape (rectangle in this case)
    kernel = cv2.getStructuringElement(
        cv2.MORPH_RECT, (kernel_size, kernel_size))
    # Dilate the image using the kernel and given number of iterations
    dilated_img = cv2.dilate(img, kernel, iterations=iterations)
    # Return the dilated image
    return dilated_img


def find_regions(img):
    output = img
    h, w = img.shape[:2]
    m = 1  # initialize the region counter to 1

    # loop through all pixels in the image
    for y in range(h):
        for x in range(w):
            # if the pixel is white (value of 255)
            if img[y][x] == 255:
                # flood fill the region and label it with the current region counter
                output = floodfill(img, x, y, m)
                # increment the region counter
                m += 1

    # loop through all the labeled regions and print their label and size
    for em in range(m):
        print('region_label: m:', em, 'count:', np.count_nonzero(output == em))

    return output


def floodfill(img, x, y, m=2):
    # Get the height and width of the image
    h, w = img.shape[:2]
    # Create a queue to store the pixels to be filled
    # And add the starting pixel to the queue
    Q = []
    Q.append((x, y))

    # Process pixels in the queue until the queue is empty
    while len(Q) > 0:
        # Remove the next pixel from the queue
        x, y = Q.pop()
        # Check that the pixel is within the image bounds and is not already filled
        if (0 <= x < w) and (0 <= y < h) and (img[y][x] == 255):
            # Fill the pixel with the specified value
            img[y][x] = m
            # Add the neighboring pixels to the queue
            Q.append((x+1, y))
            Q.append((x, y+1))
            Q.append((x-1, y))
            Q.append((x, y-1))
    # Return the filled image
    return img


def color_regions(img):
    # Get the height and width of the input image
    h, w = img.shape[:2]
    # Create an output image with the same height and width as the input image, and 3 color channels
    output = np.zeros((h, w, 3), dtype=np.uint8)

    # Loop through each pixel in the input image
    for y in range(h):
        for x in range(w):
            if (img[y][x] > 0):  # If the pixel value is greater than 0, it is part of a region
                # Get the region number of the pixel and subtract 1 (region numbers start from 1)
                m = img[y][x] - 1
                # Set the color of the pixel in the output image to the color of the corresponding region
                output[y][x] = colors[m]
    # Return the colored image
    return output

def find_contours(img):
    contours, hierarchy = cv2.findContours(img, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    return contours, hierarchy

def draw_contours(img, contours):
    img = cv2.drawContours(img, contours, -1, color=(255, 255, 255), thickness=5)
    return img


In [35]:
img = cv2.imread("../Utils/Beispiel_Tilman2.jpg")

img = binarize_image(bgr_to_gray(img), otsu_threshold(img))

img = dilate(dilate(erode(erode(img))))

c, h = find_contours(img)

out = np.zeros_like(img)

show_image(draw_contours(out, c), 0.2)

#img = find_regions(img)

#show_image(img, 0.2)


In [ ]:
show_image(color_regions(img), 0.2)

: 